**Студент: Разуев Г.А.**
<br>
**Группа:  КЭ - 403**

# Задание

1. Разработайте программу, которая выполняет классификацию заданного набора данных с помощью одной из техник ансамблевой классификации. Параметрами программы являются набор данных, ансамблевая техника (бэггинг, случайный лес или бустинг), количество участников ансамбля, а также параметры в соответствии с выбранной техникой ансамблевой классификации.
2. Проведите эксперименты на наборе данных из задания Классификация с помощью дерева решений, варьируя количество участников ансамбля (от 50 до 100 с шагом 10).
3. Выполните визуализацию полученных результатов в виде следующих диаграмм:
    * показатели качества классификации в зависимости от количества участников ансамбля для заданного набора данных; * нанесите на диаграмму соответствующие значения, полученные в задании Классификация с помощью дерева решений.
4. Подготовьте отчет о выполнении задания и загрузите отчет в формате PDF в систему. Отчет должен представлять собой связный и структурированный документ


# 1. Реализация алгоритма

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from typing import List
import os
import time

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
from sklearn.model_selection import train_test_split  # Импортируем функцию для разделения данных
from sklearn.tree import DecisionTreeClassifier, plot_tree    # Импортируем класс для построения дерева решений
from sklearn.metrics import classification_report, confusion_matrix  # Импортируем функции для оценки модели
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from typing import Any, Tuple, Literal, get_args  # Импортируем Any для аннотации типов данных

In [82]:
EnsembleTechniques = Literal['bagging', 'random_forest', 'boosting']


class EnsembleModel:
    _technique_estimators = {
        'bagging': BaggingClassifier, 'random_forest': RandomForestClassifier, 'boosting': GradientBoostingClassifier
    }

    def __init__(self, technique: EnsembleTechniques='random_forest',  random_state: int = 42, **kwargs) -> None:
        """
        Инициализация класса DecisionTreeModel.

        :param data: Входной DataFrame с данными
        :param target_column: Название колонки с целевой переменной
        :param criterion: Критерий разбиения для дерева решений, по умолчанию 'gini'
        """
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None

        self.preprocessed_X_train = None
        self.preprocessed_X_test = None

        self.accuracy = None
        self.precision = None 
        self.recall = None
        self.f1_score = None

        self.data_preprocessor = None

        clf = self._technique_estimators.get(technique) # Инициализация классификатора
        self.classifier = clf(**kwargs)
        self.random_state = random_state  # Сохранение случайного состояния для воспроизводимости

    def fit(self,X: pd.DataFrame, y: pd.Series, test_size: float = 0.2, verbose: bool = False, sample_weight=None) -> DecisionTreeClassifier:
        """
        Обучение модели на обучающей выборке.
        
        :param test_size: Доля тестовой выборки
        :param random_state: Случайное состояние для воспроизводимости
        """
        X = X.copy()
        y = y.copy()
                    

        if test_size == 0:
            self.X_train = X
            self.y_train = y

            data_preprocessor, cat_cols = self._create_data_preporcessor(X)
            preprocessed_X = self._preprocess_data(X, data_preprocessor, cat_cols)
            self.data_preprocessor = data_preprocessor
            self.preprocessed_X_train = preprocessed_X
            
            self.classifier.fit(preprocessed_X, y, sample_weight=sample_weight)
            return self.classifier
        
        # Разделяем данные на обучающую и тестовую выборки
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=self.random_state)
        data_preprocessor, cat_cols = self._create_data_preporcessor(X_train)

        preprocessed_X_train = self._preprocess_data(X_train, data_preprocessor, cat_cols, fitted=False)
        preprocessed_X_test = self._preprocess_data(X_test, data_preprocessor, cat_cols, fitted=True)

        self.data_preprocessor = data_preprocessor
        self.preprocessed_X_train = preprocessed_X_train
        self.preprocessed_X_test = preprocessed_X_test

        # Обучаем модель на обучающей выборке
        self.classifier.fit(preprocessed_X_train, y_train, sample_weight=sample_weight)

        # Делаем предсказания на тестовой выборке
        y_pred = self.classifier.predict(preprocessed_X_test)

        self.accuracy = accuracy_score(y_test, y_pred)
        self.precision = precision_score(y_test, y_pred, average='binary') 
        self.recall = recall_score(y_test, y_pred, average='binary')
        self.f1_score = f1_score(y_test, y_pred, average='binary')

        if verbose:
            print("\nОтчет о классификации:")
            print(classification_report(y_test, y_pred))

    def predict(self, X: pd.DataFrame) -> pd.Series:
        """
        Прогнозирование на основе новой выборки.

        :param new_data: Набор данных для прогнозирования
        :return: Предсказанные классы
        """
        return self.classifier.predict(X)

    def _create_data_preporcessor(self, data) -> Tuple[ColumnTransformer, List[str]]:
        num_columns = data.select_dtypes(include=np.number).columns
        cat_columns = data.select_dtypes(include="object").columns

        num_transformer = Pipeline(steps=[("scaler", StandardScaler())])
        cat_transformer = Pipeline(steps=[("OHE", OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

        data_preprocessor = ColumnTransformer(transformers=[("num_transformer", num_transformer, num_columns),
                                                            ("cat_transformer", cat_transformer, cat_columns)],
                                            remainder="passthrough")
        return data_preprocessor, cat_columns
    
    def _preprocess_data(self, data, data_preprocessor, cat_columns, fitted: bool = False) -> pd.DataFrame:
        if fitted:
            preprocessed_data = data_preprocessor.transform(data)
        else:
            preprocessed_data = data_preprocessor.fit_transform(data)
        # return preprocessed_data
        # print(type(preprocessed_data))
        # print(cat_columns)

        new_num_names = data_preprocessor.transformers_[0][2].copy()  # Копируем имена числовых колонок
        if len(cat_columns) != 0:
            new_cat_names = data_preprocessor.named_transformers_['cat_transformer'].get_feature_names_out(cat_columns)
            new_names = np.concatenate((new_num_names, new_cat_names))
        else:
            new_names = np.array(new_num_names)
        # print(preprocessed_data.shape)
        preprocessed_data = pd.DataFrame(preprocessed_data, columns=new_names)
        return preprocessed_data

    def _preprocess_splitted_X(self, X_train, X_test):
        train_preprocessor, train_cat_columns = self._create_data_preporcessor(X_train)
        X_train_preprocessed = self._preprocess_data(X_train, train_preprocessor, train_cat_columns)
        self.preprocessed_X_train =  X_train_preprocessed

        test_preprocessor, test_cat_columns = self._create_data_preporcessor(X_test)
        X_test_preprocessed = self._preprocess_data(X_test, test_preprocessor, test_cat_columns)
        self.preprocessed_X_test =  X_test_preprocessed

        return X_train_preprocessed, X_test_preprocessed



In [6]:
from sklearn.datasets import load_iris  # Импортируем набор данных Ирис
iris = load_iris()  

X = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # Создаем DataFrame
y = pd.Series(iris.target)  # Добавляем целевую переменную

In [7]:
pd.concat([X, y], axis=1).rename(columns={0: 'target'}).head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [11]:
# Создаем экземпляр класса DecisionTreeModel с критерием 
model = EnsembleModel(technique='bagging')
# Обучение модели
model.fit(X, y, verbose=True)


Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [14]:
# Создаем экземпляр класса DecisionTreeModel с критерием 
model = EnsembleModel(technique='random_forest')
print(type(model.classifier))
# Обучение модели
model.fit(X, y, verbose=True)

<class 'sklearn.ensemble._forest.RandomForestClassifier'>

Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [15]:
# Создаем экземпляр класса DecisionTreeModel с критерием 
model = EnsembleModel(technique='boosting')
print(type(model.classifier))
# Обучение модели
model.fit(X, y, test_size=0.7, verbose=True)

<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>

Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.90      0.85      0.88        33
           2       0.85      0.91      0.88        32

    accuracy                           0.92       105
   macro avg       0.92      0.92      0.92       105
weighted avg       0.92      0.92      0.92       105



# 2. Эксперименты

 ## Загрузка данных 



Данные были извлечены из базы данных бюро переписи населения, доступной по адресу: [census.gov](http://www.census.gov/ftp/pub/DES/www/welcome.html). 

- **Количество экземпляров**: 48,842 (обучающая выборка: 32,561; тестовая выборка: 16,281)
- **Целевая задача**: Предсказать, зарабатывает ли человек более 50K в год.
- **Вероятности классов**:
  - Для метки '>50K': 23.93%
  - Для метки '<=50K': 76.07%

## Признаки:

1. **age**: непрерывный
2. **workclass**: 
   - Private
   - Self-emp-not-inc
   - Self-emp-inc
   - Federal-gov
   - Local-gov
   - State-gov
   - Without-pay
   - Never-worked
3. **fnlwgt**: непрерывный (конечный вес)
4. **education**: 
   - Бакалавр
   - Некоторый колледж
   - 11 класс
   - Высшее образование
   - Профессиональная школа
   - Ассоциированный (академический и профессиональный)
   - 9 класс
   - 7-8 класс
   - 12 класс
   - Магистр
   - 1-4 класс
   - 10 класс
   - Докторская степень
   - 5-6 класс
   - Детский сад
5. **education-num**: непрерывный
6. **marital-status**: 
   - Женат (гражданский супруг)
   - Разведен
   - Никогда не женат
   - Раздельно живущий
   - Вдова
   - Женат, супруг отсутствует
   - Женат на супруге военнослужащего
7. **occupation**: 
   - Техническая поддержка
   - Ремесло
   - Другие услуги
   - Продажи
   - Исполнительный менеджер
   - Профессиональная специальность
   - Уборщики
   - Диспетчеры
   - Административный работник
   - Сельское хозяйство и рыбалка
   - Транспорт и перемещение
   - Обслуживание частных домов
   - Защита
   - Военные
8. **relationship**: 
   - Жена
   - Ребенок
   - Муж
   - Не в семье
   - Другой родственник
   - Не состоящий в браке
9. **race**: 
   - Белый
   - Азиатско-Тихоокеанский островитянин
   - Американец-эскимос
   - Другие
   - Черный
10. **sex**: 
    - Женский
    - Мужской
11. **capital-gain**: непрерывный
12. **capital-loss**: непрерывный
13. **hours-per-week**: непрерывный
14. **native-country**: Страны с различными названиями (например, США, Канада, Германия и др.)

## Изменения в данных:
- Данные были дискретизированы по доходу с порогом в 50,000.
- Ненужные символы были заменены (например, "U.S." на "US").
- Пропущенные значения заменены на "Unknown".

## Примечания:
Данные были извлечены в 1994 году на основе условий для выборки, чтобы избежать отсутствующих и конфликтующих записей.

In [16]:
DATA_PATH = 'data'
data_columns = [
    'age',               # Возраст (непрерывный)
    'workclass',        # Класс работы
    'fnlwgt',           # Конечный вес (непрерывный)
    'education',        # Образование
    'education-num',    # Номер образования (непрерывный)
    'marital-status',   # Семейное положение
    'occupation',       # Занятость
    'relationship',     # Родственные отношения
    'race',             # Раса
    'sex',              # Пол
    'capital-gain',     # Прибыль от капитала (непрерывный)
    'capital-loss',      # Убыток от капитала (непрерывный)
    'hours-per-week',   # Часы работы в неделю (непрерывный)
    'native-country',    # Страна происхождения
    'target'            # Доход (целевой признак: '>50K' или '<=50K')
]

In [17]:
train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'),  header=None)
train.columns = data_columns

In [18]:
train.shape

(32561, 15)

In [19]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [20]:
train.isna().sum().sum()

0

In [21]:
train.target = train.target.str.strip().replace({'<=50K': 0, '>50K': 1})

C:\Users\george\AppData\Local\Temp\ipykernel_8748\2278690674.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.target = train.target.str.strip().replace({'<=50K': 0, '>50K': 1})


In [22]:
pd.merge(
    train.target.value_counts().to_frame().reset_index(),
    train.target.value_counts(normalize=True).mul(100).round(2).to_frame().reset_index(),
    on='target'
)


,target,count,proportion
0,0,24720,75.92
1,1,7841,24.08


In [23]:
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'),  header=None, skiprows=1)
test.columns = data_columns

In [24]:
test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [25]:
test.shape

(16281, 15)

In [26]:
test.target.unique()

array([' <=50K.', ' >50K.'], dtype=object)

In [27]:
test.target = test.target.str.strip().replace({'<=50K.': 0, '>50K.': 1})

C:\Users\george\AppData\Local\Temp\ipykernel_8748\167773820.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test.target = test.target.str.strip().replace({'<=50K.': 0, '>50K.': 1})


In [28]:
pd.merge(
    test.target.value_counts().to_frame().reset_index(),
    test.target.value_counts(normalize=True).mul(100).round(2).to_frame().reset_index(),
    on='target'
)

,target,count,proportion
0,0,12435,76.38
1,1,3846,23.62


In [29]:
full_df = pd.concat([train, test])

In [30]:
full_df.shape

(48842, 15)

## Эксперименты с разными параметрами

In [35]:

from tqdm.autonotebook import tqdm

C:\Users\george\AppData\Local\Temp\ipykernel_8748\3512084794.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [84]:
techniques = list(get_args(EnsembleTechniques))

# Хранение результатов
results = {technique: {"accuracy": [], "precision": [], "recall": [], "f1_score": []} for technique in techniques}
n_estimators = list(range(50, 101, 10))

In [89]:
for technique in techniques:
    for n_estimator in tqdm(n_estimators, desc=f'Estimators for {technique}', leave=True):
        model = EnsembleModel(technique, n_estimators=n_estimator)
        # _data = full_df.head(1000)
        # model.fit(_data.drop(columns=['target']), _data['target'], test_size=0.3)
        model.fit(full_df.drop(columns=['target']), full_df['target'], test_size=0.3)

        # Сохранение метрик
        results[technique]["accuracy"].append(model.accuracy)
        results[technique]["precision"].append(model.precision)
        results[technique]["recall"].append(model.recall)
        results[technique]["f1_score"].append(model.f1_score)

Estimators for boosting: 100%|██████████| 6/6 [01:31<00:00, 15.31s/it]


In [80]:
# Функция для создания графиков
def create_plot(x, y_data, title, y_label):
    fig = go.Figure()
    for label, data in y_data.items():
        fig.add_trace(go.Scatter(x=x, y=data, mode='lines+markers+text', name=label, text=list(map(lambda x: f'{x:.2f}', data))))

    fig.update_layout(title=title,
                      xaxis_title='N Estimators',
                      yaxis_title=y_label,
                      legend_title='Criterion',
                      template='plotly_white')
    
    return fig

In [86]:
results

{'bagging': {'accuracy': [0.8166666666666667,
   0.8166666666666667,
   0.8233333333333334,
   0.8266666666666667,
   0.8166666666666667,
   0.82],
  'precision': [0.6129032258064516,
   0.6129032258064516,
   0.625,
   0.639344262295082,
   0.6060606060606061,
   0.6229508196721312],
  'recall': [0.5507246376811594,
   0.5507246376811594,
   0.5797101449275363,
   0.5652173913043478,
   0.5797101449275363,
   0.5507246376811594],
  'f1_score': [0.5801526717557252,
   0.5801526717557252,
   0.6015037593984962,
   0.6,
   0.5925925925925926,
   0.5846153846153846]},
 'random_forest': {'accuracy': [0.8233333333333334,
   0.8166666666666667,
   0.8333333333333334,
   0.8366666666666667,
   0.8333333333333334,
   0.8333333333333334],
  'precision': [0.6428571428571429,
   0.6296296296296297,
   0.6666666666666666,
   0.6785714285714286,
   0.6610169491525424,
   0.6507936507936508],
  'recall': [0.5217391304347826,
   0.4927536231884058,
   0.5507246376811594,
   0.5507246376811594,
   0.5

In [90]:
accuracy_fig = create_plot(n_estimators, {k: results[k]['accuracy'] for k in results}, 'Accuracy', 'Accuracy')
accuracy_fig.show()

In [91]:
accuracy_fig = create_plot(n_estimators, {k: results[k]['precision'] for k in results}, 'Precision', 'Precision')
accuracy_fig.show()

In [92]:
# Показатель качества: Recall
recall_fig = create_plot(n_estimators, {k: results[k]['recall'] for k in results}, 'Recall', 'Recall')
recall_fig.show()

In [93]:
# Показатель качества: F1 Score
f1_fig = create_plot(n_estimators, {k: results[k]['f1_score'] for k in results}, 'F1 Score', 'F1 Score')
f1_fig.show()

In [94]:
compare_accuracy = {k: max(results[k]['accuracy']) for k in results}

# Показатель, расчитанный в предыдущей работе
compare_accuracy['tree'] = 0.82

compare_accuracy

{'bagging': 0.8568211287790896,
 'random_forest': 0.8537500853067631,
 'boosting': 0.8680816215109534,
 'tree': 0.82}

In [95]:
# Извлечение ключей и значений из словаря
techniques = list(compare_accuracy.keys())
scores = list(compare_accuracy.values())

# Создание фигуры с использованием Plotly
fig = go.Figure(data=[go.Bar(x=techniques, y=scores, text=[f"{score:.2f}" for score in scores])])

# Настройка оформления графика
fig.update_layout(
    title='Comparison of Techniques',
    xaxis_title='Technique',
    yaxis_title='Accuracy',
    yaxis=dict(range=[0.7, 1]),  # Устанавливаем диапазон оси Y от 0 до 1
    template='plotly_white',  # Выбор шаблона оформления
)

# Отображение графика
fig.show()